In [17]:
import pandas as pd
import numpy as np
import requests

In [18]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [19]:
request = requests.get(url)
json = request.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [20]:
print(request)

<Response [200]>


In [21]:
#Create & save as a csv a master list of all EPL players
master_player_df = pd.DataFrame(json['elements'])
master_player_df
#Ensure we can read the whole dataframe, without "..."
pd.set_option("display.max_rows", None)
print(master_player_df.dtypes)
master_player_df.head()


chance_of_playing_next_round            float64
chance_of_playing_this_round            float64
code                                      int64
cost_change_event                         int64
cost_change_event_fall                    int64
cost_change_start                         int64
cost_change_start_fall                    int64
dreamteam_count                           int64
element_type                              int64
ep_next                                  object
ep_this                                  object
event_points                              int64
first_name                               object
form                                     object
id                                        int64
in_dreamteam                               bool
news                                     object
news_added                               object
now_cost                                  int64
photo                                    object
points_per_game                         

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,0.0,0.0,438098,0,0,-1,1,0,3,0.0,...,169,106,627,286,627,286,591,259,0.0,0.00
1,50.0,0.0,205651,0,0,-2,2,0,4,0.5,...,38,17,648,69,648,69,203,33,0.0,0.00
2,NaN,NaN,226597,0,0,0,0,0,2,5.7,...,82,5,43,8,47,9,33,11,1.0,0.67
3,NaN,NaN,219847,0,0,1,-1,1,4,8.3,...,12,4,9,3,9,3,17,5,1.0,0.67
4,0.0,0.0,463748,0,0,0,0,0,1,0.0,...,625,64,411,50,411,50,525,64,0.0,0.00


In [22]:
#element_type = position ID. The IDs are as follows:
    # 1-Goalkeeper, # 2-Defender, 3-Midfielder, 4-Forward
    
master_player_df_names = master_player_df[["id","first_name", "second_name", "team", "element_type", "code"]]
master_player_csv = master_player_df_names.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/DFQ_scratch_work/final_work/master_player.csv", index=False)
#pd.set_option("display.max_rows", None)
master_player_df_names.head()

,id,first_name,second_name,team,element_type,code
0,1,Fábio,Ferreira Vieira,1,3,438098
1,2,Gabriel,Fernando de Jesus,1,4,205651
2,3,Gabriel,dos Santos Magalhães,1,2,226597
3,4,Kai,Havertz,1,4,219847
4,5,Karl,Hein,1,1,463748


In [23]:
#Screen out the goalies only
#element_type = position ID. The IDs are as follows:
    # 1-Goalkeeper, # 2-Defender, 3-Midfielder, 4-Forward
goalie_name_list = master_player_df_names[master_player_df_names["element_type"]==1]
goalie_name_list.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/DFQ_scratch_work/final_work/goalie_name_list.csv", index=False)

In [24]:
list_of_teams_df = pd.DataFrame(json['teams'])
list_of_teams_df
#Ensure we can read the whole dataframe, without "..."
pd.set_option("display.max_rows", None)
list_of_teams_df[["id", "name"]]
list_of_teams_df.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_2/data/processed/team_name_list_and_id.csv", index=False)


In [25]:
#Create a function to ID historical stats for individual players

def get_past_data(element_id):
    url_past = f'https://fantasy.premierleague.com/api/element-summary/{element_id}/'
    response = requests.get(url_past)
    past_data = response.json()
    history_past_df = pd.DataFrame(past_data['history_past'])
    
    return history_past_df

#Get individual players' historical data using their ID
element_id = 2
history_past_df = get_past_data(element_id)

# Seee all columns
pd.set_option("display.max_columns", None)
history_past_df


,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2016/17,205651,90,91,67,651,7,4,3,5,0,0,0,2,0,0,12,258,335.8,165.9,436.0,92.9,0,0.00,0.00,0.00,0.00
1,2017/18,205651,105,104,126,1660,13,7,9,12,0,0,2,6,0,0,18,473,603.6,360.3,1089.0,204.9,0,0.00,0.00,0.00,0.00
2,2018/19,205651,105,99,79,1017,7,3,2,11,0,0,0,1,0,0,7,259,395.2,300.5,864.0,156.3,0,0.00,0.00,0.00,0.00
3,2019/20,205651,95,99,146,2018,14,8,12,14,0,0,1,3,0,0,17,535,757.6,447.7,1620.0,282.1,0,0.00,0.00,0.00,0.00
4,2020/21,205651,95,91,115,2056,9,5,11,17,0,0,0,2,0,0,16,453,501.0,467.5,931.0,190.0,0,0.00,0.00,0.00,0.00
5,2021/22,205651,85,86,120,1871,8,8,11,15,0,0,0,1,0,0,16,488,567.4,530.9,1126.0,222.3,0,0.00,0.00,0.00,0.00
6,2022/23,205651,80,81,125,2064,11,7,9,23,0,0,0,6,0,0,17,401,660.6,427.8,1331.0,241.9,24,14.24,2.91,17.15,23.15
7,2023/24,205651,80,77,85,1470,4,7,11,11,0,0,0,6,0,0,10,271,364.8,403.0,756.0,152.7,17,6.27,3.26,9.53,11.67
